In [1265]:
import numpy as np
import os,sys
import configparser
import argparse

In [1266]:
parser = argparse.ArgumentParser(description='test')
parser.add_argument('--m', type=str, choices=['LU','QR','H','G','URV'],default='LU')
parser.add_argument('--input', type=str, default='../data/LU.txt')
parser.add_argument('--solve',type=int, default=0)
parser.add_argument('--vector',type=str, default='../data/V_LU.txt')

config = configparser.ConfigParser()
config.read('../config/config.ini')
arg_list = []

#default: LU test
for k,v in config['URV_test'].items():
    arg_list.append("--" + k)
    arg_list.append(v)
args = parser.parse_args(arg_list)
print(args.solve)

0


In [1267]:
def read_matrix(file):
    f = open(file, 'r',encoding='utf-8')
    lines = f.readlines()
    matrix = []
    m = len(lines)
    l = lines[0].strip('\n').split(' ')
    n = len(l)
    i= 0    
    matrix = np.zeros((m, n), dtype=float)
    for line in lines:
        l = line.strip('\n').split(' ')
        matrix[i: ] = l[0:n]
        i += 1
    f.close()
    return matrix


In [1268]:
def read_vector(file):
    f = open(file, 'r',encoding='utf-8')
    lines = f.readlines()
    line = lines[0].strip('\n').split(' ')
    n = len(line)
    vector = np.zeros(n,dtype=float)
    vector[0:] = line[0:n]
    f.close()
    return(vector)

In [1269]:
def LU_Fact(mat):

    n = len(mat)
    interchange = 1
    P = np.eye(len(mat))

    for k in range(n):
        m_abs = np.abs(mat[k:,k])
        pivot = np.max(m_abs)
        if (pivot != mat[k,k]):
            p_index = np.where(m_abs == pivot)[0] + k
            P[p_index],P[k] = P[k],P[p_index]
            mat[p_index],mat[k] = mat[k],mat[p_index]
            interchange = -interchange
        for i in range(k+1,n):
            factor = mat[i][k] / mat[k][k]
            mat[i,k] = factor 
            for j in range(k+1,n):
                mat[i,j] = mat[i,j] - factor * mat[k,j]

    L = np.tril(mat)
    U = np.triu(mat)

    det_U = 1
    for i in range(n):
        L[i,i] = 1;
        det_U = det_U * U[i,i]
    
    det = det_U * interchange
    return P,L,U,det

In [1270]:
def detQ(Q):#solve det(Q) in QR factorization
    n = Q.size()
    I = np.eye(n)
    r = np.linalg.matrix_rank(-I-Q)#to judge det(Q)
    n = np.linalg.matrix_rank(Q)
    if ((n-r) % 2 == 1):
        det = -1
    else:
        det = 1 
    return det

In [1271]:
def detR(R):#solve det(R) in QR factorization
    n = R.size()
    det_R = 1
    for i in range(n):
        det_R = det_R * R[i,i]
    return det_R

In [1272]:
def QR_Fact(mat):
    m,n = mat.shape
    Q = np.zeros((m,min(m,n)))
    R = np.zeros((min(m,n),n))
    for k in range(n):
        Q[:,k] = mat[:,k]
        for i in range(0,k):
            R[i,k] = Q[:,i].T.dot(mat[:,k])
            #print("R:",i,k,R[i,k],"\n")
            Q[:,k] = Q[:,k] - R[i,k] * Q[:,i]
        R[k,k] = np.linalg.norm(Q[:,k])
        #print("R:",k,k,R[k,k],"\n")
        Q[:,k] = Q[:,k] / R[k,k]
        #print("Q:",k,Q[:,k],"\n")
    
    if m == n:
        det = detR(R) * detQ(Q)
    else:
        det = "Error"
    return Q,R,det

In [1273]:
def Householder(mat):
    m,n = mat.shape
    Q = np.eye(m)
    R = mat.copy()
    if (m <= n):
        size = m - 1
    else:
        size = n
    for i in range(0,size):
        e = np.zeros_like(R[i:,i])
        I = np.eye(m-i)
        a= np.eye(m)
        e[0] = 1
        mod = np.linalg.norm(R[i:,i])
        u = R[i:,i] - mod * e;
        #print(u)
       
        v1 = u.reshape(len(u),1)
        v2 = u.reshape(1,len(u))
        frac = v2.dot(v1)  
        a[i:,i:] = I - (2/frac) * v1.dot(v2)
        #print(a)
        R = a.dot(R)
        Q = Q.dot(a)
        #print("Q: ", Q,"\n")

    if m == n:
        det = detR(R) * detQ(Q)
    else:
        det = "Error"
    return Q,R,det

In [1274]:
def Givens(mat):
    m,n = mat.shape
    Q = np.eye(m)
    R = mat.copy()
    for i in range(0,n):
        for j in range(i+1,m):
            P = np.eye(m)
            x = np.hypot(R[j,i],R[i,i])
            print(R[i,i],R[j,i])
            print(x)
            cos = R[i,i]/x
            sin = R[j,i]/x
            P[j,i] = -sin
            P[i,j] = sin
            P[i,i] = cos
            P[j,j] = cos
            #print(a)
            R = P.dot(R)
            Q = P.dot(Q)
            #print("Q: ", Q,"\n")

    Q = Q.T
    if m == n:
        det = detR(R) * detQ(Q)
    else:
        det = "Error"
    return Q,R,det

In [1275]:
def URV(mat):
    m,n = mat.shape
    r = np.linalg.matrix_rank(mat)
    Q,B_1,_ = Householder(mat)
    B = B_1[:r,:]
    print(B.T)
    q,B_2,_ = Householder(B.T)
    T = B_2[:r,:r]
    V = q
    U = Q
    R = np.zeros_like(mat)
    R[:r,:r] = T.T
    if m == n:
        det = detR(R) * detQ(U) * detQ(V.T)
    else:
        det = "Error"
    return U,R,V,det

In [1276]:
def Solver_U(A,y):
    m,n = A.shape
    if m!=n:#consistency
        print("This system does not have a certain solution.\n")
        x = []
        return x
    x = np.zeros(n)
    for i in range(n-1,-1,-1):
        sum = 0
        for j in range(i+1,n):
            sum += A[i][j] * x[j]
        x[i] = (y[i] - sum) / A[i][i]
    
    return x

In [1277]:
def Solver_L(A,y):
    x = np.zeros(n)
    for i in range(0,n):
        sum = 0
        for j in range(0,i):
            sum += A[i][j] * x[j]
        x[i] = (y[i] - sum) / A[i][i]
    
    return x

In [1278]:
mat = read_matrix(args.input)
if(mat.size <= 0):
    print("Input is not a matrix\n")
    sys.exit(1)
print("Input Matrix:")
print(mat)
#print(np.linalg.det(mat))
#print(np.linalg.qr(mat))
if (args.m == 'LU'):
    print("Your choice is : LU factorization")
    m,n = mat.shape
    if (m != n):
        print("Your input matrix should be a square matrix\n")
        sys.exit(1)
    P,L,U,det = LU_Fact(mat)
    print("P:")
    print(P)
    print("L:")
    print(L)
    print("U:")
    print(U)
    print("det：")
    print(det)
    if(args.solve == 1):
        b = read_vector(args.vector)
        b = P.dot(b)
        y = Solver_L(L,b)
        x = Solver_U(U,y)
        print("The reslut of Ax = b is: ")
        print(x)

Input Matrix:
[[-4. -2. -4. -2.]
 [ 2. -2.  2.  1.]
 [-4.  1. -4. -2.]]


In [1279]:
if (args.m == 'QR'):
    print("Your choice is : Gram-Schmidt QR factorization")
    rank = np.linalg.matrix_rank(mat)
    m,n = mat.shape
    if(rank !=m and rank !=n):#Gram-Schmidt得不到正定矩阵
        print("This matrix cannot be QR factorized by Gram-Schmidt")
    Q,R,det = QR_Fact(mat)
    print("Q:")
    print(Q)
    print("R:")
    print(R)
    if(det == "Error"):
        print("Your matrix must be square to calculate determinant\n")
    else:
        print("det：")
        print(det)
    if(args.solve == 1):
        b = read_vector(args.vector)
        b = Q.T.dot(b)
        x = Solver_U(R,b)
        print("The reslut of Ax = b is: ")
        print(x)

In [1280]:
if (args.m == 'H'):
    print("Your choice is : Householder QR factorization")
    m,n = mat.shape
    Q,R,det = Householder(mat)
    print("Q:")
    print(Q)
    print("R:")
    print(R)
    if(det == "Error"):
        print("Your matrix must be square to calculate determinant\n")
    else:
        print("det：")
        print(det)
    if(args.solve == 1):
        b = read_vector(args.vector)
        b = Q.T.dot(b)
        x = Solver_U(R,b)
        print("The reslut of Ax = b is: ")
        print(x)

In [1281]:
if (args.m == 'G'):
    print("Your choice is : Givens QR factorization")
    m,n = mat.shape
    Q,R,det = Givens(mat)
    print("Q:")
    print(Q)
    print("R:")
    print(R)
    if(det == "Error"):
        print("Your matrix must be square to calculate determinant\n")
    else:
        print("det：")
        print(det)
    if(args.solve == 1):
        b = read_vector(args.vector)
        b = Q.T.dot(b)
        x = Solver_U(R,b)
        print("The reslut of Ax = b is: ")
        print(x)

In [1282]:
if (args.m == 'URV'):
    print("Your choice is : URV factorization")
    m,n = mat.shape
    U,R,V,det = URV(mat)
    print("U:")
    print(U)
    print("R:")
    print(R)
    print("V: ")
    print(V)
    if(det == "Error"):
        print("Your matrix must be square to calculate determinant\n")
    else:
        print("det：")
        print(det)
    if(args.solve == 1):
        b = read_vector(args.vector)
        b = U.T.dot(b)
        R = R.dot(V.T)
        x = Solver_U(R,b)
        print("The reslut of Ax = b is: ")
        print(x)

Your choice is : URV factorization
[[ 6.00000000e+00 -3.55271368e-16]
 [ 2.22044605e-16  3.00000000e+00]
 [ 6.00000000e+00 -3.55271368e-16]
 [ 3.00000000e+00 -1.77635684e-16]]
U:
[[-0.66666667 -0.66666667 -0.33333333]
 [ 0.33333333 -0.66666667  0.66666667]
 [-0.66666667  0.33333333  0.66666667]]
R:
[[ 9.00000000e+00 -1.23259516e-32  0.00000000e+00  0.00000000e+00]
 [-4.58892184e-16  3.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
V: 
[[ 6.66666667e-01  2.46716228e-17 -6.66666667e-01 -3.33333333e-01]
 [ 2.46716228e-17  1.00000000e+00  4.93432455e-17  2.46716228e-17]
 [ 6.66666667e-01 -4.93432455e-17  7.33333333e-01 -1.33333333e-01]
 [ 3.33333333e-01 -2.46716228e-17 -1.33333333e-01  9.33333333e-01]]
Your matrix must be square to calculate determinant

